<a href="https://colab.research.google.com/github/Manjuphoenix/ML_team_buzz/blob/michelle/VGG16_custom_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt

import scipy
from PIL import Image
from scipy import ndimage

import torch
import torchvision

from torch import utils
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, transforms
import torchvision.models as models

import copy
import time

# Steps to add data in kaggle:
# 1. Click on Add data, and 
# if os.path('/kaggle/working/test'):
#     print("Skipping the transfer of data")
# else:
!mkdir /content/test
!mkdir /content/train

mkdir: cannot create directory ‘/content/test’: File exists
mkdir: cannot create directory ‘/content/train’: File exists


In [6]:
#train_x_orig, train_y, test_x_orig, test_y, classes = load_data()

!pip install kaggle

from google.colab import files 
files.upload()

#Make a directory named kaggle and copy the kaggle.json file there; and change its permissions
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

#imagenet_data = torchvision.datasets.ImageNet('https://kaggle.com/saroz014/plant-diseases')
#data_loader = torch.utils.data.DataLoader(imagenet_data, batch_size=4, shuffle=True, num_workers=args.nThreads)

#choose location of kaggle.json
#C:\Users\micht\.kaggle

Saving kaggle.json to kaggle (1).json


In [3]:
!kaggle datasets download -d saroz014/plant-diseases

100% 6.39G/6.39G [01:04<00:00, 112MB/s] 
100% 6.39G/6.39G [01:04<00:00, 106MB/s]


In [4]:
from zipfile import ZipFile
file_name = 'plant-diseases.zip' #the file is your dataset exact name
with ZipFile(file_name, 'r') as zip:
  #zip.printdir()
  zip.extractall()
  #zip.close()
  print('Done')

Done


In [7]:
import os
from torch import utils
import shutil

dir_path = '/content/dataset_itr2/dataset_itr2/test'
cl = os.listdir(dir_path)
print(cl)
unique_plants = []
for i in cl:
    x = i.split('_')
    if x[0] not in unique_plants:
        unique_plants.append(x[0])
print("Number of Unique Plants: ",len(unique_plants))
print("Unique Plants: ",unique_plants)

#What I wanna do here, is seperate the data based on their unique classes
#Creating Directories in another folder to avoid confusion, so mypath is like the target path:
my_path = '/content/test'
for u in unique_plants:
    new_dir = my_path + '/' + u
    if os.path.isdir(new_dir) == False:
        os.mkdir(new_dir)
        print(new_dir + ' -> Directory created')

#moving data into them
new_dirs = os.listdir(my_path)
#Note : cl holds the original directories as segregated by disease
for i in cl:
    x = i.split('_')
    for d in new_dirs:
        if x[0] == d:
            #check if folder already exists  
            target = my_path + '/' + d
            if os.path.isdir(target) == True:
                shutil.move(dir_path + '/' + i, target)



dir_path = '/content/dataset_itr2/dataset_itr2/train'
cl = os.listdir(dir_path)
print(cl)
unique_plants = []
for i in cl:
    x = i.split('_')
    if x[0] not in unique_plants:
        unique_plants.append(x[0])
print("Number of Unique Plants: ",len(unique_plants))
print("Unique Plants: ",unique_plants)

#What I wanna do here, is seperate the data based on their unique classes
#Creating Directories in another folder to avoid confusion, so mypath is like the target path:
my_path = '/content/train'
for u in unique_plants:
    new_dir = my_path + '/' + u
    if os.path.isdir(new_dir) == False:
        os.mkdir(new_dir)
        print(new_dir + ' -> Directory created')

#moving data into them
new_dirs = os.listdir(my_path)
#Note : cl holds the original directories as segregated by disease
for i in cl:
    x = i.split('_')
    for d in new_dirs:
        if x[0] == d:
            #check if folder already exists  
            target = my_path + '/' + d
            if os.path.isdir(target) == True:
                shutil.move(dir_path + '/' + i, target)

['Apple___Apple_scab', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Grape___Black_rot', 'Raspberry___healthy', 'Pepper,_bell___healthy', 'Apple___Cedar_apple_rust', 'Tomato___Leaf_Mold', 'Corn_(maize)___Northern_Leaf_Blight', 'Grape___Esca_(Black_Measles)', 'Apple___Black_rot', 'Strawberry___healthy', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Pepper,_bell___Bacterial_spot', 'Tomato___Septoria_leaf_spot', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Potato___Early_blight', 'Tomato___Early_blight', 'Tomato___Target_Spot', 'Cherry_(including_sour)___healthy', 'Potato___healthy', 'Grape___healthy', 'Squash___Powdery_mildew', 'Potato___Late_blight', 'Corn_(maize)___Common_rust_', 'Tomato___healthy', 'Soybean___healthy', 'Strawberry___Leaf_scorch', 'Cherry_(including_sour)___Powdery_mildew', 'Corn_(maize)___healthy', 'Tomato___Bacterial_spot', 'Tomato___Tomato_mosaic_virus', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Apple___healthy', 'Blueberry___healthy', 'Peach___healt

In [9]:
from fastai.callbacks.hooks import num_features_model

#define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
PATH = 'model.pth' # to save model

#load data
data_transforms = {

    'train/Cherry' : transforms.Compose([
        transforms.RandomResizedCrop(256),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test/Cherry': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(256),#
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

    ])
}
data_dir = '/content'

#create custom dataset
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train/Cherry', 'test/Cherry']}


dataloader = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size = 4, shuffle = True, num_workers = 2) for x in ['train/Cherry', 'test/Cherry']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train/Cherry', 'test/Cherry']}

class_names = image_datasets['train/Cherry'].classes
#print(class_names)
#['Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Target_Spot', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Tomato___Tomato_mosaic_virus', 'Tomato___healthy']



In [10]:
#Training

def train_model(model, criterion, optimizer, scheduler, num_epochs=2):
  since = time.time() #to know time taken to train model
  best_model_wts = copy.deepcopy( model.state_dict()) #saving model weights
  best_acc = 0.0

  for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch, num_epochs-1))
    print("-" *18)

    #num of epochs for training n validation phase
    for phase in ['train/Cherry', 'test/Cherry']:
      if phase == 'train/Cherry':
        model.train() #to train
      else:
        model.eval() #to evaluate model
      
      running_loss = 0.0
      running_corrests = 0

      ##Iterating through data
      for inputs, labels in dataloader[phase]:
        inputs = inputs.to(device)
        labels = labels.to(device)

        #zero-th parameter gradients
        optimizer.zero_grad()

        # forward propagation
        # track history of training
        with torch.set_grad_enabled(phase == 'train/Cherry'):
          outputs = model(inputs)
          _, preds = torch.max(outputs, 1)
          loss = criterion(outputs, labels)

          # backward + optimize only if in training phase
          if phase == 'train/Cherry':
            loss.backward()
            optimizer.step()

        #generating some stats
        running_loss += loss.item() * inputs.size(0)
        running_corrests += torch.sum(preds == labels.data)

      if phase == 'train/Cherry':
        scheduler.step()

      epoch_loss = running_loss / dataset_sizes[phase]
      epoch_acc = running_corrests.double() / dataset_sizes[phase]

      print(' {} Loss: {:.4f} \t Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

      # Deep copy the model
      if phase == 'test/Cherry' and epoch_acc > best_acc:
        best_model_wts = copy.deepcopy(model.state_dict())
        torch.save(model.state_dict(), "./best-model-checkpoint.pt")
        best_acc = epoch_acc

    print()

  time_elapsed = time.time() - since
  print("Training Completed in time {:.0f}m {:.0f}s ".format(time_elapsed//60, time_elapsed % 60))
  print('Best val Acc: {:4f}'.format(best_acc))

  #loading best model weights
  model.load_state_dict(best_model_wts)
  return model


In [ ]:
# 2. Create the neural network layers
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1),
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1))
        self.layer2 = nn.AdaptiveAvgPool2d(output_size=(4096))
        self.layer3 = nn.Sequential(
                nn.Linear(in_features=57344, out_features=4096, bias=True),
                nn.ReLU(),
                nn.Dropout(p=0.5),
                nn.Linear(in_features=4096, out_features=10, bias=True),
            )

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        return out


model_ft = NeuralNetwork()


model_ft = model_ft.to(device)

#Loss function
criterion = nn.CrossEntropyLoss()
#define optimizer
optimizer_ft = optim.Adam(model_ft.parameters(), lr = 0.001)

#define scheduler
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size = 7, gamma = 0.1)

model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs = 2)

torch.save(model_ft.state_dict(), './my2epochmodel.pth')

print('\nSaving the model...')
torch.save(model_ft, PATH)

print('Model Saved!')


Epoch 0/1
------------------
